GRUPO: 

# Práctica 2

Vamos a ver en la práctica los conceptos vistos en clase sobre Recomendadores.
- En la primera parte veremos algunos conceptos de la librería Pandas de python ejemplificados en lo que hemos llamado Recomendador Simple, que ordena y muestra las películas en el TOP de popularidad pero no realiza recomendaciones personalizadas, es decir, a todos los usuarios les recomienda la misma lista de películas (como en el TOP 10 de Netflix).
- En la segunda parte veremos un recomendador personalizado basado en contenidos y 
- En la tercera parte veremos los recomendadores por filtrado colaborativo usando la librería SURPRISE y haciendo hincapié en como evaluar los resultados de la recomendación. 


## Primera parte. Recomendacion simple por popularidad

Esta primera parte está inspirada en Rounak Banik, 2018 Recommender Systems in Python: Beginner Tutorial DE DATACAMP.COM
Vamos a utilizar los dos datasets de MovieLens que se pueden descargar de: 
https://www.kaggle.com/rounakbanik/the-movies-dataset

- The Full Dataset: Consists of 26,000,000 ratings and 750,000 tag applications applied to 45,000 movies by 270,000 users. Includes tag genome data with 12 million relevance scores across 1,100 tags.

- The Small Dataset: Comprises of 100,000 ratings and 1,300 tag applications applied to 9,000 movies by 700 users.

En esta primera parte solo tienes que ejecutar las celdas del tutorial y responder las preguntas del final. No se pide implementar nada extra

### Recomendador simple basado en popularidad

Vamos a realizar una version simple de un Recomendador que ofrece recomendaciones generalizadas a todos los usuarios según la popularidad de la película y el género. La idea básica detrás de este recomendador es que las películas que son más populares y más aclamadas por la crítica tendrán una mayor probabilidad de gustar a la audiencia promedio. Este modelo no da recomendaciones personalizadas en base a los gustos de un usuario concreto.

La implementación de este modelo es muy sencilla. Basta ordenar las películas según las calificaciones y la popularidad y mostrar las mejores películas de nuestra lista. Como paso adicional, podemos pasar un argumento de género para filtrar y mostrar solo las mejores películas de un género en particular. 

Los pasos que vamos a realizar en esta primera versión son los siguientes:
 - Decidir cual es el criterio que nos permite ordenar las peliculas recomendadas. 
 - Calcular esa métrica para cada pelicula de la lista total de películas.
 - Ordenar las películas y mostrar las mejores.

Cargamos los datos del archivo csv de peliculas usando pandas.
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

Pandas es un paquete de Python que proporciona estructuras de datos tabulares con columnas de tipo heterogéneo con etiquetas en columnas y filas. Estas tablas se conocen como DataFrames. Un dataframe es una estructura de datos con dos dimensiones en la cual se puede guardar datos de distintos tipos (como caractéres, enteros, valores de punto flotante, factores y más) en columnas. Es similar a una hoja de cálculo o una tabla de SQL

In [1]:
# Importamos Pandas
import pandas as pd
import warnings; 
warnings.simplefilter('ignore')  # para evitar algunos warnings al cargar los datos.

# Load Movies Metadata in a dataframe
md = pd.read_csv('./data/movies_metadata.csv', low_memory=False)

# se puede imprimir solo las primeras 3 filas para ver como son los datos.
md.head(3)
#md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


Utilizo las clasificaciones de The Movie Database (TMDb)  para crear nuestra tabla de películas principales. 
Vamos a utilizar la fórmula de calificación ponderada de IMDB Internet Movie Database.

Calificación ponderada o Weighted Rating (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$
dónde,
- v es el número de votos para la película
- m son los votos mínimos requeridos para aparecer en la tabla
- R es la calificación promedio de la película
- C es el voto medio

El siguiente paso es determinar un valor apropiado para m, los votos mínimos requeridos para aparecer en la tabla. Usaremos el percentil 95 como nuestro límite. En otras palabras, para que una película aparezca en las listas, debe tener más votos que al menos el 95% de las películas de la lista.

Vamos a hacer una tabla Top 250 por popularidad y otra para seleccionar las películas de un genero en particular.  


In [2]:
from ast import literal_eval
import numpy as np
import matplotlib.pyplot as plt

In [3]:
## obtenemos la media de las puntuaciones de las películas de la tabla. 
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [5]:
# cuantas cumplen el percentil 0,95
m = vote_counts.quantile(0.95)
m

434.0

Para que una película sea elegida tiene que tener por los menos 434 votos en TMDB (95%). Tambien vemos que la media de rating de las peliculas en TMDB es de 5.244 en una escala de 10. Hay 2274 peliculas que cumplen estos criterios.

In [6]:
# Utilizo la columna release_date para construir una columna year, porque solo me interesa el año. 
# No es necesario comprender todos los detalles pero puedes consultar la documentación de panda.

md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [7]:
# limpiamos la lista de generos para que se quede solo con el nombre en vez de la estructura [{'id': 16, 'name': 'Animation'}. Sólo ejecutar una vez. 
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

Vamos a mostrar el resultado despues de modificar la columna genres y añadir la columna year para el año. 

In [8]:
md.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995


In [10]:

seleccionadas = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
seleccionadas['vote_count'] = seleccionadas['vote_count'].astype('int')
seleccionadas['vote_average'] = seleccionadas['vote_average'].astype('int')
seleccionadas.shape
# shape devuelve el numero filas y columnas de la tabla de peliculas seleccionadas. 

(2274, 6)

In [11]:
seleccionadas.head(3)

,title,year,vote_count,vote_average,popularity,genres
0,Toy Story,1995,5415,7,21.946943,"[Animation, Comedy, Family]"
1,Jumanji,1995,2413,6,17.015539,"[Adventure, Fantasy, Family]"
5,Heat,1995,1886,7,17.924927,"[Action, Crime, Drama, Thriller]"


In [12]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [13]:
seleccionadas['wr'] = seleccionadas.apply(weighted_rating, axis=1)

In [14]:
seleccionadas = seleccionadas.sort_values('wr', ascending=False).head(250)

###  TOP 15 de peliculas

In [15]:
seleccionadas.head(15)
#mostramos el TOP 15

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


Vemos que tres películas de Christopher Nolan:  **Inception**, **The Dark Knight** e **Interstellar** son las preferidas por popularidad en la parte superior de nuestra lista de TOP 15. En la tabla se ve un fuerte sesgo de los usuarios de TMDB hacia géneros y directores particulares, pero de los sesgos no hablaremos en esta práctica. 

Vamos a construir una funcion que permite filtrar y construir una tabla de peliculas recomendadas para generos particulares. 

In [16]:
# Separamos las peliculas que tienen varios generos de forma que aparezcan varias filas de la misma película.
# una por cada genero.
# De nuevo no hace falta entender todos los detalles. Sólo observar el comportamiento en la tabla resultado.

s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [17]:
gen_md.head(3)

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Family


In [18]:
# Metemos todo el código anterior en una función.
# Vamos a relajar nuestras condiciones predeterminadas al percentil 85 en lugar de 95 para tener más generos.
    
def construye_tabla(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    seleccionadas = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    seleccionadas['vote_count'] = seleccionadas['vote_count'].astype('int')
    seleccionadas['vote_average'] = seleccionadas['vote_average'].astype('int')
    
    seleccionadas['wr'] = seleccionadas.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    seleccionadas = seleccionadas.sort_values('wr', ascending=False).head(250)
    
    return seleccionadas

Veamos nuestro método en acción mostrando las 15 mejores películas románticas. Con esto forzamos el genero que nos interesa y resolvemos el problema de que el romance casi no figuraba en nuestro Top  genérico a pesar de ser uno de los géneros de películas más populares.

### TOP Películas románticas

In [19]:
construye_tabla('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457024,8.565285
351,Forrest Gump,1994,8147,8,48.307194,7.971357
876,Vertigo,1958,1162,8,18.20822,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988,834,8,14.177005,7.744878
19901,Paperman,2012,734,8,7.198633,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.994281,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


Hasta aquí lo único que hemos hecho es ordenar el catalogo. Las recomendaciones son generales (no personalizadas) por popularidad, pero lo que nosotros queremos hacer son recomendaciones personalizadas para un usuario del que sabemos su perfil, sus preferencias, alguna película que le ha gustado.
La siguiente parte corresponde al recomendador basado en contenido que dada una película (puede ser usada como perfil) recomienda peliculas similares. En la tercera parte veremos el filtrado colaborativo.

Para el Recomendador simple hemos usado las películas del dataset completo, mientras que para el sistema de recomendación personalizado haremos uso del segundo data set (Small)

### Ejercicio parte 1.

Realiza algunas pruebas de recomendaciones por popularidad y comenta los resultados obtenidos.  Incluye los resultados y los comentarios en este archivo.

¿Te parece útil esta forma de recomendación de contenidos? 
¿Sería útil también para otras aplicaciones, por ejemplo, para recomendar libros o restaurantes?




In [ ]:
# qué pruebas hacer? qué comentar sobre ellas?

In [20]:
# y responder a la última pregunta

## Segunda parte.  Recomendacion basada en contenido


El recomendador que construimos en la sección anterior da la misma recomendación a todos, independientemente del gusto personal del usuario. Si una persona que le gustan las películas románticas (y odia la acción) mirara nuestro Top 15, probablemente no le gustarían la mayoría de las películas. Si esta persona mira nuestras listas por género, no estaría recibiendo las mejores recomendaciones.

Por ejemplo, considere a una persona que le gustan: "Dilwale Dulhania Le Jayenge", "My Name is Khan" y "Kabhi Khushi Kabhi Gham". Si miramos los datos de estas películas podríamos inferir que a esta persona le gusta el actor Shahrukh Khan y el director Karan Johar. Incluso si él / ella tuviera acceso a la tabla de películas románticas no las encontraría como las principales recomendaciones, porque no son películas populares. 

Para personalizar más nuestras recomendaciones, voy a crear un motor que calcule la similitud entre películas en función de ciertas métricas y sugiera películas que son más similares a una película en particular que le gustó a un usuario. Como hemos visto en teoría este tipo de recomendaciones se conocen como **recomendación o filtrado basado en contenido**.

Vamos a ver cómo construir dos recomendadores basados en contenido:

- Usando los resúmenes textuales de las películas y taglines. Los taglines son las frases cortas que  suelen aparecer en el cartel de la película, acompañando al título y a una imagen destacada y representativa de la historia que presenta.
- Usando el reparto de películas, equipo, palabras clave y género



In [21]:
# usaremos el conjunto pequeño de películas por eficiencia. 
# Leemos los datos y limpiamos algunos valores nulos.
links_small = pd.read_csv('./data/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')
## selecciono las peliculas de links_small con los id de la tabla de metadatos 
smd = md[md['id'].isin(links_small)]
smd.shape
## shape devuelve el numero de filas y columnas. 
## (9099, 25)
# Tenemos *9099* películas disponibles en nuestro conjunto de datos de metadatos de películas pequeñas, que es 5 veces más pequeño que nuestro conjunto de datos original de 45000 películas.

(9099, 25)

In [22]:
smd.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995


Tenemos que medir la similitud de las películas con las preferencias del usuario. En este caso usaremos como preferencia una pelicula que le haya gustado antes para recuperar (por similitud) otras películas parecidas.  Elegimos usar similitud usando el atributo de descripción (textual) y el lema (tambien textual). Para calcular la similitud entre dos textos usamos el modelo del espacio vectorial.

In [23]:
smd['tagline'] = smd['tagline'].fillna('')
#El método fillna permite sustituir los valores nulos de una estructura pandas

smd['description'] = smd['overview'] + smd['tagline']
# creamos un campo llamado descrition con la 
smd['description'] = smd['description'].fillna('')

In [24]:
smd.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,description
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,A family wedding reignites the ancient feud be...


In [25]:
print(smd['description'])

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
40224    From the mind behind Evangelion comes a hit la...
40503    The band stormed Europe in 1963, and, in 1964,...
44821    When Molly Hale's sadness of her father's disa...
44826    All your favorite Pokémon characters are back,...
45265    While holidaying in the French Alps, a Swedish...
Name: description, Length: 9099, dtype: object


#### Similitud del coseno 

Utilizamos el vectorizador TF-IDF
Para calcular un valor de similitud se puede usar **cosine_similarity** o **linear_kernel** de sklearn. La segunda es más eficiente. 
https://scikit-learn.org/stable/modules/metrics.html

La medida de similitud del coseno calcula un valor numerico que representa la similitud entre dos vectores de términos. En este caso los vectores representan los textos de la descripción normalizados.

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [27]:
tfidf_matrix.shape
# filas, columnas

(9099, 268124)

In [28]:
#cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [29]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

Ahora tenemos una matriz de similitud de coseno por pares para todas las películas en nuestro conjunto de datos. El siguiente paso es escribir una función que devuelva las 30 películas más similares según la puntuación de similitud del coseno.

In [30]:
smd = smd.reset_index() 
#reset_index resets the index column) to a regular column and uses a numeric sequence as new index.

titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])
#Un objeto "Series" es un vector con datos indexados

In [31]:
indices.head(3)

title
Toy Story           0
Jumanji             1
Grumpier Old Men    2
dtype: int64

In [32]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [33]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

###  Ejercicio parte 2: 
Prueba algunas consultas y comenta los resultados. Compara los resultados con el tipo de recomendaciones realizadas en la parte 1. 
    
    